# Basic Usage

In [ ]:
# this cell is hidden from output
# it’s used to 
# - import credentials
# - hide the warnings that are displayed because 
#   of a current mismatch in geos versions between 
#   shapely and pygeos
# - set sys.path to point to the local streetviewdownloader repo

import yaml
import os.path
with open(os.path.expanduser("~/.config/streetviewdownloader.yaml")) as _config_file:
    _config = yaml.safe_load(_config_file)

API_KEY = _config["API_KEY"]
URL_SIGNING_KEY = _config["URL_SIGNING_KEY"]
del _config

import warnings
warnings.simplefilter("ignore")

import sys
sys.path.append("../src/")

In [ ]:
import shapely.wkt
import ipyleaflet

from streetviewdownloader import StreetViewDownloader

First, let’s define where to save the data, and for which area we want to download data:

In [ ]:
output_directory = "/home/${USER}/Downloads/StreetView/"
kirkkonummi = shapely.wkt.loads("POLYGON((24.40 60.12, 24.45 60.11, 24.46 60.13, 24.40 60.12))")

map1 = ipyleaflet.Map()
map1.add_layer(ipyleaflet.WKTLayer(wkt_string=kirkkonummi.wkt))
map1.fit_bounds(((kirkkonummi.bounds[1],kirkkonummi.bounds[0]),(kirkkonummi.bounds[3],kirkkonummi.bounds[2])))
map1

In [ ]:
from streetviewdownloader import StreetNetworkPointGenerator, StreetViewDownloader, StreetViewMetadataDownloader
points = StreetNetworkPointGenerator().points_on_street_network(kirkkonummi)
points.head()

In [ ]:
map2 = ipyleaflet.Map()
map2.add_layer(ipyleaflet.WKTLayer(wkt_string=kirkkonummi.wkt))
map2.add_layer(ipyleaflet.GeoData(geo_dataframe=points, point_style={"radius": 1, "color": "black"}))
map2.center = map1.center
map2.zoom = map1.zoom + 2

map2

In [ ]:
metadata = StreetViewMetadataDownloader(API_KEY, URL_SIGNING_KEY).download(points)

In [ ]:
metadata.head()

In [ ]:
map3 = ipyleaflet.Map()
map3.add_layer(ipyleaflet.WKTLayer(wkt_string=kirkkonummi.wkt))
map3.add_layer(ipyleaflet.GeoData(geo_dataframe=metadata[["geometry"]], point_style={"radius": 0.5, "color": "black" }))
map3.center = map1.center
map3.zoom = map1.zoom + 2
map3

The 